<h1>Clean and prepare subject data</h1>
<p>Data preparation consists of:
    <ul>
        <li>Nuisance regression</li>
        <li>Group mask generation</li>
        <li>ISC mask generation</li>
        <li>Data normalization</li>
        <li>Trimming blank TRs and offsetting to account for hemodyamic lag
    </ul>
</li>

In [1]:
#### Imports
import os 
import numpy as np
import data_prep_helpers as helpers

# Notebook specific imports
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
%autosave 5

/opt/miniconda3/envs/mybrainiak/lib/python3.7/site-packages/nilearn/datasets/__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


Autosaving every 5 seconds


<h2>Set subject groups</h2>
<p>
    <ul>
        <li><b>Subject IDs for 3 year olds:</b> 6, 8, 9, 11, 12, 13, 14, 16, 17, 19, 20, 22, 23, 25, 26, 27, 28</li>
        <li><b>Subject IDs for 4 year olds:</b> 1, 2, 3, 4, 5, 7, 10, 15, 18, 21, 24, 29, 30 31</li>
        <li><b>Subject IDs for 5 year olds:</b> 32-65</li>
        <li><b>Subject IDs for 7 year olds:</b> 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 79, 80, 81, 83, 84, 86, 88, 96, 99, 114, 116, 122</li>
        <li><b>Subject IDs for 8-12 year olds:</b> 77, 78, 82, 85, 87, 89, 90, 91, 92, 93, 94, 95, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 115, 117, 118, 119, 120, 121</li>
        <li><b>Subject IDs for adults:</b> 123-155</li>       
    </ul>
</p>

In [63]:
# Modify this section to change subject group.
subject_group_str = '3-4-yrs'
subject_ids = range(1,32)

# Modify this section to set data location
directory = os.path.join(os.path.expanduser('~'), 'Documents', 'Data', subject_group_str)
print('Data location:', directory)

Data location: /Users/angira/Documents/Data/3-4-yrs


<h2>BOLD data</h2>
<p> Either load raw data and apply nuisance regression, or load pre-cleaned data</p>

<h3>Load and clean non-cleaned data</h3>

In [ ]:
# Clean data
cleaned_data = []
for subject in subject_ids: 
    subject_cleaned = helpers.clean_subject_data(subject, directory, save=True)
    cleaned_data.append(subject_cleaned)

print('Cleaned data:', np.shape(cleaned_data))

<h3>Load previously cleaned data</h3>

In [64]:
cleaned_data = helpers.load_clean_data(subject_ids, os.path.join(directory, 'raw_subjects'))
print('Cleaned data:', np.shape(cleaned_data))

Cleaned data: (31,)


<h2>Group mask</h2>
<p>Either generate group mask by downsampling MNI152 template from nilearn, or load previously generated mask</p>

<h3>Generate mask</h3>

In [ ]:
from nilearn import image as niimage

group_mask = niimage.resample_img(img=datasets.load_mni152_brain_mask(),
                                  target_affine=cleaned_data[0].affine,
                                  target_shape=(79, 95, 68))

print('Group mask:', np.shape(group_mask))

<h3>Load existing group mask</h3>

In [7]:
from nibabel import load as load_mask

group_mask = load_mask(os.path.join(os.path.expanduser('~'), 'Documents', 'Data', 'group_mask.nii.gz'))
print('Group mask:', np.shape(group_mask))

Group mask: (79, 95, 68)


<h2>ISC masking</h2>
Either peform ISC or load previously performed ISC, and apply as mask to BOLD data

<h3>Perform ISC and thresholding</h3>

In [40]:
# Generate ISC maps
isc_maps = helpers.perform_isc(cleaned_data, group_mask)
print('Successfully performed ISC, shape:', np.shape(isc_maps))

Successfully performed ISC, shape: (34, 377996)


In [41]:
# Threshold ISC
thresholded_isc, num_significant_voxels = helpers.threshold_isc(isc_maps, threshold=0.05)
print('%d significant voxels after FDR controlling at threshold %f' %(num_significant_voxels, 0.05))

153687 significant voxels after FDR controlling at threshold 0.050000


In [42]:
# Save unthresholed ISC values
np.save(os.path.join(directory, (subject_group_str +'_isc.npy')), isc_maps)
# Save thresholded ISC values
np.save(os.path.join(directory, (subject_group_str +'_thresholded_isc.npy')), thresholded_isc)

<h3>Load previously thresholded ISC</h3>

In [40]:
thresholded_isc = np.squeeze(np.load(os.path.join(directory, (subject_group_str + '_isc_thresholded.npy'))))
print('Thresholded ISC:', np.shape(thresholded_isc))

Thresholded ISC: (377996,)


<h3>Use adult ISC</h3>

In [65]:
thresholded_isc = np.squeeze(np.load(os.path.join(os.path.expanduser('~'), 
                                                  'Documents', 
                                                  'Data', 
                                                  'adults', 'adults_isc_thresholded.npy')))
print('Thresholded ISC:', np.shape(thresholded_isc))

Thresholded ISC: (377996,)


<h3>Apply ISC mask to data</h3>

In [66]:
# Apply ISC mask to cleaned data
masked_data = helpers.apply_isc_mask(cleaned_data, thresholded_isc, group_mask)
print(np.shape(masked_data))

(31, 136543, 168)


<h2>Normalize, trim, and shift data</h2>

In [67]:
# Trim blank TRs and shift for hemodynamic lag
trimmed_data = helpers.trim_blank_trs(masked_data)

print('Trimmed data size:', np.shape(trimmed_data))

# Apply normalization 
normalized_data = helpers.normalize_data(trimmed_data)

print('Normalized data size:', np.shape(normalized_data))

Trimmed data size: (31, 136543, 152)
Normalized data size: (31, 136543, 152)


In [68]:
# Save normalized data as numpy array
# np.save(os.path.join(directory, (subject_group_str +'_normalized_data.npy')), normalized_data)
np.save(os.path.join(directory, (subject_group_str +'_normalized_data_adult_mask.npy')), normalized_data)

print('Successfully saved normalized data')

Successfully saved normalized data
